In [1]:
import torch
import pathlib
import random
import json
from tqdm.auto import tqdm
import more_itertools
import numpy as np
import pandas as pd

/Users/keisuke.ogaki/.ghq/github.com/Hi-king/learn_slack_emoji_completion/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('..')
import emojicompletion

In [3]:
tokenizer = emojicompletion.data.Tokenizer()
candidates, case_dict = emojicompletion.data.SlackEmojiCompletionDataset(
    directory=pathlib.Path('..') /
    'data').load(filter_by_vocabulary=True)
candidates = list(candidates)


In [4]:
model = emojicompletion.model.SimpleLSTM(
    n_token=len(tokenizer.dictionary),
    output_type='last',
)

model.load_state_dict(torch.load('../results/adaptivelstm_batch50_lr0.0001_commitea975c7_hardnegTrue_20220706084401/model_epoch400.pth'))
model = model.eval()

params = json.load(open('../results/2input_l2_lstm_batch50_lr0.001_commitea975c7_hardnegFalse_20220706103635/params.json'))
keys_test = params['keys_test']

In [7]:
all_df = pd.DataFrame(index=candidates)
for key in tqdm(keys_test[:100]):
    result = []
    for xs_str in more_itertools.chunked(candidates, n=100):
        with torch.inference_mode():
            maxlen = max(len(x) for x in xs_str)

            xs = torch.stack(
                [
                    tokenizer.tokenize(key + '/' + x + "*" * (maxlen - len(x)))
                    for x in xs_str
                ],
                dim=-1,
            )
            pred = torch.nn.Sigmoid()(model(xs)).to('cpu').numpy()
            for i in range(len(pred)):
                score = pred[i,0]
                result.append(dict(key=xs_str[i], score=score))
    df = pd.DataFrame(result)
    df = df.sort_values(by="score", ascending=False).assign(rank=range(1, len(df)+1))
    all_df = all_df.assign(**{key: df.set_index("key")["rank"]})

100%|██████████| 100/100 [01:43<00:00,  1.04s/it]


In [8]:
n = 0
ranks = 0
for key in all_df.columns:
    targets = case_dict[key]
    n += len(targets)
    ranks += all_df.loc[targets, key].sum()
print(n, ranks, ranks/n)


82 860 10.487804878048781
